In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt


import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import my_model

Load in data

In [2]:
def show_statistic(l):
    max_value = max(l)
    min_value = min(l)
    mean = sum(l)/len(l)
    var = sum([((x - mean) ** 2) for x in l]) / len(l)
    std = var**0.5
    print("min:", min_value)
    print("max:", max_value)
    print("mean:", mean)
    print("std:", std)

def normalization(l):
    max_value = max(l)
    min_value = min(l)
    mean = sum(l)/len(l)
    var = sum([((x - mean) ** 2) for x in l]) / len(l)
    std = var**0.5
    # print("min:", min_value)
    # print("max:", max_value)
    # print("mean:", mean)
    # print("std:", std)
    
    for i in range(len(l)):
        l[i] = (l[i] - mean)/std
    
    return l
    
    
def min_max_scaling(l):
    max_value = max(l)
    min_value = min(l)
    for i in range(len(l)):
        l[i] = (l[i] - min_value)/(max_value - min_value)
    
    return l
    

def load_data(file_name, test_size):
    f = open(file_name)
    df = pd.read_csv(f)
    data = np.array(df[['x_0','x']])
    x = data[:,0].tolist()
    x.append(data[-1, -1])
    # do normalization
    show_statistic(x)
    x = min_max_scaling(x)
    show_statistic(x)
    
    data = np.array(df[['y_0','y']])
    y = data[:,0].tolist()
    y.append(data[-1, -1])
    # do normalization
    y = min_max_scaling(y)

    data = np.array(df[['z_0','z']])
    z = data[:,0].tolist()
    z.append(data[-1, -1])
    # do normalization
    z = min_max_scaling(z)
    
    train_set_x = x[:-test_size]
    test_set_x = x[-test_size:]
    train_set_y = y[:-test_size]
    test_set_y = y[-test_size:]
    train_set_z = z[:-test_size]
    test_set_z = z[-test_size:]
    
    return train_set_x, test_set_x, train_set_y, test_set_y, train_set_z, test_set_z

def normalize_all(d_1, d_2, d_3, d_4, d_5, l_1, l_2, l_3, l_4, l_5):
    d_all = d_1 + d_2 + d_3 + d_4 + d_5
    # get mean and std of all traning data
    mean = sum(d_all)/len(d_all)
    var = sum([((x - mean) ** 2) for x in d_all]) / len(d_all)
    std = var**0.5
    # do normalization for each training data set
    for i in range(len(d_1)):
        d_1[i] = (d_1[i] - mean)/std
    for i in range(len(d_2)):
        d_2[i] = (d_2[i] - mean)/std
    for i in range(len(d_3)):
        d_3[i] = (d_3[i] - mean)/std
    for i in range(len(d_4)):
        d_4[i] = (d_4[i] - mean)/std
    for i in range(len(d_5)):
        d_5[i] = (d_5[i] - mean)/std
        
    # do normalization for each test data set by using mean and std from traning data sets
    for i in range(len(l_1)):
        l_1[i] = (l_1[i] - mean)/std
    for i in range(len(l_2)):
        l_2[i] = (l_2[i] - mean)/std
    for i in range(len(l_3)):
        l_3[i] = (l_3[i] - mean)/std
    for i in range(len(l_4)):
        l_4[i] = (l_4[i] - mean)/std
    for i in range(len(l_5)):
        l_5[i] = (l_5[i] - mean)/std
    # return d_1, d_2, d_3, d_4, d_5, l_1, l_2, l_3, l_4, l_5
    return mean, std

In [3]:
train_set_x_1, test_set_x_1, train_set_y_1, test_set_y_1, train_set_z_1, test_set_z_1 = load_data('data_preprocessing/test_1_training_xyz.txt', 400)
train_set_x_2, test_set_x_2, train_set_y_2, test_set_y_2, train_set_z_2, test_set_z_2 = load_data('data_preprocessing/test_2_training_xyz.txt', 400)
train_set_x_3, test_set_x_3, train_set_y_3, test_set_y_3, train_set_z_3, test_set_z_3 = load_data('data_preprocessing/test_3_training_xyz.txt', 400)
train_set_x_4, test_set_x_4, train_set_y_4, test_set_y_4, train_set_z_4, test_set_z_4 = load_data('data_preprocessing/test_4_training_xyz.txt', 400)
train_set_x_5, test_set_x_5, train_set_y_5, test_set_y_5, train_set_z_5, test_set_z_5 = load_data('data_preprocessing/test_5_training_xyz.txt', 400)

min: -53.879546999999995
max: 55.39066700000001
mean: 1.201871477941177
std: 24.715028995911588
min: 0.0
max: 1.0
mean: 0.5040844751886471
std: 0.22618267221396277
min: -65.839828
max: 58.818431999999994
mean: 0.9881137397058822
std: 28.473596818315595
min: 0.0
max: 1.0
mean: 0.5360891587906473
std: 0.22841323806633868
min: -53.948696
max: 62.748402
mean: 4.172788362647053
std: 29.38591584163824
min: 0.0
max: 1.0
mean: 0.4980542392120766
std: 0.2518135955843416
min: -59.879162
max: 68.182877
mean: 2.104038269411762
std: 27.857703124497807
min: 0.0
max: 1.0
mean: 0.4840091626950572
std: 0.21753287189576814
min: -60.71685
max: 59.075089
mean: 1.9414046220588248
std: 27.954991497077714
min: 0.0
max: 1.0
mean: 0.5230590233793504
std: 0.23336287675481832


In [4]:
# show_statistic(train_set_x_1)
x_mean, x_std = normalize_all(train_set_x_1, train_set_x_2, train_set_x_3, train_set_x_4, train_set_x_5, test_set_x_1, test_set_x_2, test_set_x_3, test_set_x_4, test_set_x_5)
y_mean, y_std = normalize_all(train_set_y_1, train_set_y_2, train_set_y_3, train_set_y_4, train_set_y_5, test_set_y_1, test_set_y_2, test_set_y_3, test_set_y_4, test_set_y_5)
z_mean, z_std = normalize_all(train_set_z_1, train_set_z_2, train_set_z_3, train_set_z_4, train_set_z_5, test_set_z_1, test_set_z_2, test_set_z_3, test_set_z_4, test_set_z_5)
# show_statistic(train_set_x_1)

In [5]:
x_mean

0.5067009838472856

In [6]:
x_std

0.2332880326514156

In [7]:
# construct list of input and label pairs
def input_data(seq, ws):
    out = []
    L = len(seq)
    
    for i in range(L - ws):
        window = seq[i:i+ws]
        label = seq[i+ws:i+ws+1]
        out.append((window, label))
        
    return out

def get_tensor(sample_size, data_x, data_y, data_z):
    for i in range(sample_size):
        # construct dataset
        x = np.asarray(data_x[i][0]).reshape(1,-1) # 1 x window_size
        y = np.asarray(data_y[i][0]).reshape(1,-1) # 1 x window_size
        z = np.asarray(data_z[i][0]).reshape(1,-1) # 1 x window_size
        xyz = np.append(np.append(x, y, axis = 0), z, axis = 0) # 3 x window_size
        xyz = np.transpose(xyz) # window_size x 3
        window_size = xyz.shape[0]
        # print("window_size", window_size)
        xyz = xyz.reshape(1, window_size, 3) # 1 x window_size x 3
        if (i == 0):
            prev_xyz = xyz
        else:
            prev_xyz = np.append(prev_xyz, xyz, axis = 0)
        
        #construct label
        x_label = np.asarray(data_x[i][1]).reshape(1,-1) # 1 x 1
        y_label = np.asarray(data_y[i][1]).reshape(1,-1) # 1 x 1
        z_label = np.asarray(data_z[i][1]).reshape(1,-1) # 1 x 1
        xyz_label = np.append(np.append(x_label, y_label, axis = 0), z_label, axis = 0) # 3 x 1
        xyz_label = xyz_label.reshape(1, 3) # 1 x 3 x 1
        if (i == 0):
            prev_xyz_label = xyz_label
        else:
            prev_xyz_label = np.append(prev_xyz_label, xyz_label, axis = 0)
            
    return prev_xyz, prev_xyz_label

# make train_dataset
def construct_train_test_tensor(train_set_x, train_set_y, train_set_z, test_set_x, test_set_y, test_set_z, window_size):
    # sequence to data sample
    train_data_x = input_data(train_set_x, window_size)
    train_data_y = input_data(train_set_y, window_size)
    train_data_z = input_data(train_set_z, window_size)
    test_data_x = input_data(test_set_x, window_size)
    test_data_y = input_data(test_set_y, window_size)
    test_data_z = input_data(test_set_z, window_size)
    
    
    # reconstruct train/test dataset and label
    sample_size = len(train_data_x)
    train_dataset, train_label = get_tensor(sample_size, train_data_x, train_data_y, train_data_z)
    sample_size = len(test_data_x)
    test_dataset, test_label = get_tensor(sample_size, test_data_x, test_data_y, test_data_z)
    
    train_dataset = train_dataset.astype("float32")
    train_label = train_label.astype("float32")
    test_dataset = test_dataset.astype("float32")
    test_label = test_label.astype("float32")
    
    return train_dataset, train_label, test_dataset, test_label

In [8]:
window_size = 10
train_dataset_1, train_label_1, test_dataset_1, test_label_1 = construct_train_test_tensor(train_set_x_1,
                                                                                           train_set_y_1,
                                                                                           train_set_z_1,
                                                                                           test_set_x_1,
                                                                                           test_set_y_1,
                                                                                           test_set_z_1,
                                                                                           window_size)

train_dataset_2, train_label_2, test_dataset_2, test_label_2 = construct_train_test_tensor(train_set_x_2,
                                                                                           train_set_y_2,
                                                                                           train_set_z_2,
                                                                                           test_set_x_2,
                                                                                           test_set_y_2,
                                                                                           test_set_z_2,
                                                                                           window_size)

train_dataset_3, train_label_3, test_dataset_3, test_label_3 = construct_train_test_tensor(train_set_x_3,
                                                                                           train_set_y_3,
                                                                                           train_set_z_3,
                                                                                           test_set_x_3,
                                                                                           test_set_y_3,
                                                                                           test_set_z_3,
                                                                                           window_size)

train_dataset_4, train_label_4, test_dataset_4, test_label_4 = construct_train_test_tensor(train_set_x_4,
                                                                                           train_set_y_4,
                                                                                           train_set_z_4,
                                                                                           test_set_x_4,
                                                                                           test_set_y_4,
                                                                                           test_set_z_4,
                                                                                           window_size)

train_dataset_5, train_label_5, test_dataset_5, test_label_5 = construct_train_test_tensor(train_set_x_5,
                                                                                           train_set_y_5,
                                                                                           train_set_z_5,
                                                                                           test_set_x_5,
                                                                                           test_set_y_5,
                                                                                           test_set_z_5,
                                                                                           window_size)

In [9]:
train_dataset_1.dtype

dtype('float32')

In [10]:
train_dataset_1.shape

(2990, 10, 3)

In [11]:
train_label_1.shape

(2990, 3)

In [12]:
test_dataset_1.shape

(390, 10, 3)

In [13]:
test_label_1.shape

(390, 3)

In [14]:
type(train_dataset_1)

numpy.ndarray

In [15]:
# Concatenate tensors
train_dataset = np.concatenate((train_dataset_1,
                                train_dataset_2,
                                train_dataset_3,
                                train_dataset_4,
                                train_dataset_5), axis=0)
train_label = np.concatenate((train_label_1,
                              train_label_2,
                              train_label_3,
                              train_label_4,
                              train_label_5), axis=0)
test_dataset = np.concatenate((test_dataset_1,
                               test_dataset_2,
                               test_dataset_3,
                               test_dataset_4,
                               test_dataset_5), axis=0)
test_label = np.concatenate((test_label_1,
                             test_label_2,
                             test_label_3,
                             test_label_4,
                             test_label_5), axis=0)

In [16]:
train_dataset.dtype

dtype('float32')

In [17]:
train_dataset.shape

(14950, 10, 3)

In [18]:
train_label.shape

(14950, 3)

In [19]:
test_dataset.shape

(1950, 10, 3)

In [20]:
test_label.shape

(1950, 3)

In [21]:
2990*5

14950

In [22]:
390*5

1950

In [23]:
2 x 5 x 5 x 13 x 23 = 14950

SyntaxError: invalid syntax (<ipython-input-23-9979ff4634ab>, line 1)

In [24]:
2 x 5 x 5 x 13 x 3 = 1950

SyntaxError: invalid syntax (<ipython-input-24-de69bcda9ea3>, line 1)

In [25]:
2 * 5 * 5 * 13 * 23

14950

In [26]:
2 * 5 * 5 * 13 * 3

1950

In [27]:
13 * 23

299

In [28]:
13 * 3

39

In [51]:
test_dataset

array([[[ 9.8337859e-02,  2.5345819e-04,  1.0032148e+00],
        [ 1.3741747e-01,  2.5345819e-04,  1.0000477e+00],
        [ 1.7205451e-01,  2.5345819e-04,  9.8298705e-01],
        ...,
        [ 3.3839446e-01,  2.5345819e-04,  8.8670057e-01],
        [ 3.7166247e-01,  2.5345819e-04,  8.6744332e-01],
        [ 4.0493050e-01,  2.5345819e-04,  8.4818602e-01]],

       [[ 1.3741747e-01,  2.5345819e-04,  1.0000477e+00],
        [ 1.7205451e-01,  2.5345819e-04,  9.8298705e-01],
        [ 2.0532250e-01,  2.5345819e-04,  9.6372980e-01],
        ...,
        [ 3.7166247e-01,  2.5345819e-04,  8.6744332e-01],
        [ 4.0493050e-01,  2.5345819e-04,  8.4818602e-01],
        [ 4.3819851e-01,  2.5345819e-04,  8.2892871e-01]],

       [[ 1.7205451e-01,  2.5345819e-04,  9.8298705e-01],
        [ 2.0532250e-01,  2.5345819e-04,  9.6372980e-01],
        [ 2.3859046e-01,  2.5345819e-04,  9.4447249e-01],
        ...,
        [ 4.0493050e-01,  2.5345819e-04,  8.4818602e-01],
        [ 4.3819851e-01,  2.5

Prepare dataset

In [29]:
class MyTrainDataSet(Dataset):
    def __init__(self, train_dataset, train_label):
        self.train_dataset = train_dataset
        self.train_label = train_label
        
    def __len__(self):
        return len(self.train_dataset)

    def __getitem__(self, idx):
        seq = self.train_dataset[idx]
        label = self.train_label[idx]
        
        return [seq, label]


class MyTestDataSet(Dataset):
    def __init__(self, test_dataset, test_label):
        self.test_dataset = test_dataset
        self.test_label = test_label

    def __len__(self):
        return len(self.test_dataset)

    def __getitem__(self, idx):
        seq = self.test_dataset[idx]
        label = self.test_label[idx]

        return [seq, label]

In [30]:
train_set = MyTrainDataSet(train_dataset, train_label)
print(len(train_set))
valid_set = MyTestDataSet(test_dataset, test_label)
print(len(valid_set))

14950
1950


In [31]:
type(train_set.train_dataset)

numpy.ndarray

In [32]:
# batch_size = 30
# batch_size = 50
batch_size = 650
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
# keep the test data trajectory order
test_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)

In [33]:
train_loader

In [34]:
for i, (seqs, labels) in enumerate(test_loader):
    print(i, seqs.shape, labels.shape)

0 torch.Size([650, 10, 3]) torch.Size([650, 3])
1 torch.Size([650, 10, 3]) torch.Size([650, 3])
2 torch.Size([650, 10, 3]) torch.Size([650, 3])


In [35]:
for i, (seqs, labels) in enumerate(train_loader):
    if (i == 0):
        print(i, seqs.dtype, labels.dtype)
        print(seqs[0])
        print(labels[0])

0 torch.float32 torch.float32
tensor([[ 0.4985, -0.3636, -0.1193],
        [ 0.4877, -0.3636, -0.1921],
        [ 0.4770, -0.3636, -0.2649],
        [ 0.4662, -0.3636, -0.3376],
        [ 0.4555, -0.3636, -0.4104],
        [ 0.4447, -0.3636, -0.4832],
        [ 0.4339, -0.3636, -0.5560],
        [ 0.4232, -0.3636, -0.6288],
        [ 0.4124, -0.3636, -0.7015],
        [ 0.4017, -0.3636, -0.7743]])
tensor([ 0.3778, -0.3636, -0.8427])


In [36]:
for i, (seqs, labels) in enumerate(test_loader):
    if (i == 0):
        print(i, seqs.dtype, labels.dtype)
        print(seqs[1])
        print(labels[1])

0 torch.float32 torch.float32
tensor([[1.3742e-01, 2.5346e-04, 1.0000e+00],
        [1.7205e-01, 2.5346e-04, 9.8299e-01],
        [2.0532e-01, 2.5346e-04, 9.6373e-01],
        [2.3859e-01, 2.5346e-04, 9.4447e-01],
        [2.7186e-01, 2.5346e-04, 9.2522e-01],
        [3.0513e-01, 2.5346e-04, 9.0596e-01],
        [3.3839e-01, 2.5346e-04, 8.8670e-01],
        [3.7166e-01, 2.5346e-04, 8.6744e-01],
        [4.0493e-01, 2.5346e-04, 8.4819e-01],
        [4.3820e-01, 2.5346e-04, 8.2893e-01]])
tensor([4.7147e-01, 2.5346e-04, 8.0967e-01])


In [37]:
torch.cuda.is_available()

True

In [38]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first = True)
        # If your input data is of shape (seq_len, batch_size, features)
        # then you don’t need batch_first=True and your LSTM will give
        # output of shape (seq_len, batch_size, hidden_size).

        # If your input data is of shape (batch_size, seq_len, features)
        # then you need batch_first=True and your LSTM will give
        # output of shape (batch_size, seq_len, hidden_size).
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        # self.hidden = (torch.zeros(self.num_layers, 1, hidden_size), torch.zeros(self.num_layers, 1, hidden_size))
        
    def forward(self, x):
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim)).cuda()
        else:
            h0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim))
        if torch.cuda.is_available():
            c0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim)).cuda()
        else:
            c0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim))
        # print("x.size(0)", x.size(0))
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        # x is (batch_size, seq_len, features)
        
        out = self.fc(out[:, -1, :])
        
        return out
    

In [39]:
input_dim = 3
hidden_dim = 100
layer_dim = 1
output_dim = 3
model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

if torch.cuda.is_available():
    model.cuda()

In [40]:
criterion = nn.MSELoss()

In [41]:
learning_rate = 0.5
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

Check parameters

In [42]:
len(list(model.parameters()))

6

In [43]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([400, 3])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([3, 100])
torch.Size([3])


Training

In [44]:
seq_dim = 10 # = window_size

num_epochs = 1600 # 400 # 200 will overfit, 100 is good, see the plot: plt.plot(train_loss[20:]) and plt.plot(test_loss[20:])

train_loss = []
test_loss = []



for epoch in range(num_epochs):
    total_train_loss = 0.0
    train_batch = 0
    for i, (seqs, labels) in enumerate(train_loader):
        # print("train: ", i)
        if torch.cuda.is_available():
            # print(seqs.shape)
            # print(seqs[0])
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            # print(labels.shape)
            # print(seqs[0])
            # seqs = Variable(seqs)
            labels = Variable(labels.cuda())
        else:
            # print(seqs.shape)
            # print(seqs[0])
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
            # print(labels.shape)
            # print(seqs[0])
            # seqs = Variable(seqs)
            labels = Variable(labels)
        
        
        optimizer.zero_grad()
        # outputs = model(seqs.float())
        outputs = model(seqs)
        # print(outputs.is_cuda)
        # print(labels.is_cuda)
        # print(outputs.shape)
        # print(outputs.dtype)
        
        # loss = criterion(outputs, labels.float())
        loss = criterion(outputs, labels)
        total_train_loss += loss.data.item()
        
        loss.backward()
        optimizer.step()
        
        train_batch = i + 1;
        
        # print("loss: ", loss.data)
    
    total_test_loss = 0.0
    test_batch = 0
    # test_seq = []
    test_pred = []
    # test_gt = []
    for i, (seqs, labels) in enumerate(test_loader):
        # print("test: ", i)
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        outputs = model(seqs)
        # print(i, outputs.shape)
        # print(i, outputs.is_cuda)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        # test_gt.append(labels)
        test_pred.append(outputs)
        # test_seq.append(seqs)
        
        test_batch = i + 1
    
    # print("train batch: ", train_batch)
    # print("test batch: ", test_batch)
    train_loss.append(total_train_loss/train_batch)
    test_loss.append(total_test_loss/test_batch)
    # train_loss.append(total_train_loss)
    # test_loss.append(total_test_loss)
    print("epoch: {}, train_loss: {}, test_loss: {}".format(epoch, total_train_loss/train_batch, total_test_loss/test_batch))
        
        

epoch: 0, train_loss: 0.24395426451835944, test_loss: 0.024608733287702005
epoch: 1, train_loss: 0.023696140831579334, test_loss: 0.021704524134596188
epoch: 2, train_loss: 0.020620783709961434, test_loss: 0.01960700734828909
epoch: 3, train_loss: 0.018374239823416523, test_loss: 0.017711480613797903
epoch: 4, train_loss: 0.016625659013895885, test_loss: 0.016298655265321333
epoch: 5, train_loss: 0.015168875415364037, test_loss: 0.015126192631820837
epoch: 6, train_loss: 0.013941473692007687, test_loss: 0.01398835649403433
epoch: 7, train_loss: 0.012841035328481508, test_loss: 0.012928927627702555
epoch: 8, train_loss: 0.011880362851788168, test_loss: 0.011974334872017304
epoch: 9, train_loss: 0.01099638851440471, test_loss: 0.011197434893498818
epoch: 10, train_loss: 0.010193767066559067, test_loss: 0.010432820922384659
epoch: 11, train_loss: 0.009448040074304394, test_loss: 0.009733952038610974
epoch: 12, train_loss: 0.008758371554153122, test_loss: 0.009048701729625463
epoch: 13, tr

epoch: 107, train_loss: 0.0018751033500808737, test_loss: 0.0019751323270611465
epoch: 108, train_loss: 0.0018696728182713623, test_loss: 0.001965070356770108
epoch: 109, train_loss: 0.0018647004364301329, test_loss: 0.0019540356006473303
epoch: 110, train_loss: 0.0018580165671427612, test_loss: 0.0019339955373046298
epoch: 111, train_loss: 0.0018514060142004619, test_loss: 0.0019498318433761597
epoch: 112, train_loss: 0.0018444466860155048, test_loss: 0.0019251325672181945
epoch: 113, train_loss: 0.0018388819982014272, test_loss: 0.001911974938896795
epoch: 114, train_loss: 0.0018355347590682948, test_loss: 0.0019256109565806885
epoch: 115, train_loss: 0.0018265098839512339, test_loss: 0.001906927830229203
epoch: 116, train_loss: 0.0018187625303297586, test_loss: 0.0018937555335772533
epoch: 117, train_loss: 0.0018153829732909799, test_loss: 0.0018884532231216629
epoch: 118, train_loss: 0.0018094499777678561, test_loss: 0.001876070067131271
epoch: 119, train_loss: 0.001804155110300559

epoch: 211, train_loss: 0.001469572712464825, test_loss: 0.001463167155937602
epoch: 212, train_loss: 0.0014677578633975077, test_loss: 0.0014551399217452854
epoch: 213, train_loss: 0.00146418494557071, test_loss: 0.0014518068443673353
epoch: 214, train_loss: 0.0014602345738397992, test_loss: 0.001449004989505435
epoch: 215, train_loss: 0.0014586840685376007, test_loss: 0.0014463704428635538
epoch: 216, train_loss: 0.0014559272676706314, test_loss: 0.001449293427867815
epoch: 217, train_loss: 0.0014544122842261972, test_loss: 0.0014431502592439454
epoch: 218, train_loss: 0.0014522928774923735, test_loss: 0.0014299558436808486
epoch: 219, train_loss: 0.0014492900252504194, test_loss: 0.001433601777534932
epoch: 220, train_loss: 0.001444899563110717, test_loss: 0.001439917366951704
epoch: 221, train_loss: 0.0014425580308813116, test_loss: 0.001427491253707558
epoch: 222, train_loss: 0.0014428726154501023, test_loss: 0.0014303083395740639
epoch: 223, train_loss: 0.0014413458133197348, tes

epoch: 315, train_loss: 0.001264998340047896, test_loss: 0.00122917066134202
epoch: 316, train_loss: 0.001264258386546989, test_loss: 0.0012233591017623742
epoch: 317, train_loss: 0.0012619149992647378, test_loss: 0.0012173124705441296
epoch: 318, train_loss: 0.0012617042419784095, test_loss: 0.0012290706217754632
epoch: 319, train_loss: 0.0012597563909366727, test_loss: 0.0012233660963829607
epoch: 320, train_loss: 0.0012594130745603015, test_loss: 0.0012167355841180931
epoch: 321, train_loss: 0.0012569978362475724, test_loss: 0.0012154801030798505
epoch: 322, train_loss: 0.0012550907508146179, test_loss: 0.001216695944701011
epoch: 323, train_loss: 0.001254880780597096, test_loss: 0.0012094885460101068
epoch: 324, train_loss: 0.0012518162737883952, test_loss: 0.0012021438354471077
epoch: 325, train_loss: 0.0012519027475956018, test_loss: 0.0012035368320842583
epoch: 326, train_loss: 0.0012497367646099756, test_loss: 0.0012159174172362934
epoch: 327, train_loss: 0.001248964515980333, 

epoch: 419, train_loss: 0.0011377765913493931, test_loss: 0.0010794210441720982
epoch: 420, train_loss: 0.0011375169016663795, test_loss: 0.0010850625112652779
epoch: 421, train_loss: 0.0011346641037127247, test_loss: 0.0010731337048734229
epoch: 422, train_loss: 0.0011353116386327083, test_loss: 0.0010754133303028841
epoch: 423, train_loss: 0.0011338900708917367, test_loss: 0.0010884133031746994
epoch: 424, train_loss: 0.0011329189494616635, test_loss: 0.0010774564483047773
epoch: 425, train_loss: 0.0011317356535152574, test_loss: 0.001071279567743962
epoch: 426, train_loss: 0.001130877609324196, test_loss: 0.001072552113328129
epoch: 427, train_loss: 0.0011320324948705409, test_loss: 0.0010664682971158375
epoch: 428, train_loss: 0.001128832905796235, test_loss: 0.0010677027554872136
epoch: 429, train_loss: 0.0011280926992185414, test_loss: 0.0010753555203943204
epoch: 430, train_loss: 0.0011286341494111264, test_loss: 0.0010668887213493388
epoch: 431, train_loss: 0.001126094452996292

...

epoch: 1145, train_loss: 0.00079123352897232, test_loss: 0.0007125231738124663
epoch: 1146, train_loss: 0.0007902213635489992, test_loss: 0.0007300162881923219
epoch: 1147, train_loss: 0.0007904139980066406, test_loss: 0.000715636502718553
epoch: 1148, train_loss: 0.0007902160413441775, test_loss: 0.0007139719091355801
epoch: 1149, train_loss: 0.0007901324222431234, test_loss: 0.000719729796401225
epoch: 1150, train_loss: 0.0007896670266864417, test_loss: 0.0007136021498202657
epoch: 1151, train_loss: 0.0007904476774653987, test_loss: 0.0007109969107356543
epoch: 1152, train_loss: 0.0007902299175443857, test_loss: 0.0007276324856017405
epoch: 1153, train_loss: 0.0007895913339503434, test_loss: 0.0007211299719832217
epoch: 1154, train_loss: 0.0007902171776589493, test_loss: 0.000712970921692128
epoch: 1155, train_loss: 0.0007888746053061408, test_loss: 0.0007180460816016421
epoch: 1156, train_loss: 0.0007888660335953792, test_loss: 0.0007150325933859373
epoch: 1157, train_loss: 0.000789

epoch: 1247, train_loss: 0.0007679850095883012, test_loss: 0.0006911082891747355
epoch: 1248, train_loss: 0.0007689245166661947, test_loss: 0.0006983898492762819
epoch: 1249, train_loss: 0.0007690329980307623, test_loss: 0.0007044182323928302
epoch: 1250, train_loss: 0.000769019708968699, test_loss: 0.0007011956622591242
epoch: 1251, train_loss: 0.0007699619771143341, test_loss: 0.0006963227885231996
epoch: 1252, train_loss: 0.0007691136257641989, test_loss: 0.0006957789349447315
epoch: 1253, train_loss: 0.0007685102514780896, test_loss: 0.000689657720310303
epoch: 1254, train_loss: 0.000768694925405409, test_loss: 0.0006905553260973344
epoch: 1255, train_loss: 0.0007681796458833244, test_loss: 0.0006942908075870946
epoch: 1256, train_loss: 0.0007673198377470608, test_loss: 0.0006951421916407222
epoch: 1257, train_loss: 0.0007669481793013604, test_loss: 0.0006936621551479524
epoch: 1258, train_loss: 0.0007663776632398367, test_loss: 0.000690710973382617
epoch: 1259, train_loss: 0.00076

epoch: 1349, train_loss: 0.0007495902182595075, test_loss: 0.0006797244035017987
epoch: 1350, train_loss: 0.000750068484036171, test_loss: 0.0006844304443802685
epoch: 1351, train_loss: 0.0007510045113082489, test_loss: 0.0006879877570706109
epoch: 1352, train_loss: 0.0007498401505670146, test_loss: 0.0006747587855594853
epoch: 1353, train_loss: 0.0007493187529136143, test_loss: 0.0006735160714015365
epoch: 1354, train_loss: 0.0007498178367867418, test_loss: 0.0006767433612064148
epoch: 1355, train_loss: 0.0007491674927144271, test_loss: 0.0006737219955539331
epoch: 1356, train_loss: 0.0007494543970096856, test_loss: 0.0006715127237839624
epoch: 1357, train_loss: 0.0007513615773464351, test_loss: 0.0006759507523383945
epoch: 1358, train_loss: 0.0007494817862449133, test_loss: 0.0006750231890085464
epoch: 1359, train_loss: 0.0007485198844021753, test_loss: 0.0006784706347389147
epoch: 1360, train_loss: 0.0007475485617760569, test_loss: 0.0006740707952606803
epoch: 1361, train_loss: 0.00

epoch: 1451, train_loss: 0.0007367814314025252, test_loss: 0.000657123537772956
epoch: 1452, train_loss: 0.0007330042628935822, test_loss: 0.0006613396738733476
epoch: 1453, train_loss: 0.0007345669513122867, test_loss: 0.0006674452257963518
epoch: 1454, train_loss: 0.0007330877402716357, test_loss: 0.0006740519747836515
epoch: 1455, train_loss: 0.0007346272784912878, test_loss: 0.0006593580862196783
epoch: 1456, train_loss: 0.0007333839856310869, test_loss: 0.0006580703581372896
epoch: 1457, train_loss: 0.0007330159651641936, test_loss: 0.0006738184796025356
epoch: 1458, train_loss: 0.0007346454278930374, test_loss: 0.0006690450585059201
epoch: 1459, train_loss: 0.0007330951326445717, test_loss: 0.0006698967238965755
epoch: 1460, train_loss: 0.0007335060331767992, test_loss: 0.0006646478626256188
epoch: 1461, train_loss: 0.0007346820481040555, test_loss: 0.0006578946825660145
epoch: 1462, train_loss: 0.0007319301392113709, test_loss: 0.0006666292708056668
epoch: 1463, train_loss: 0.00

epoch: 1553, train_loss: 0.000720916323505504, test_loss: 0.0006473373456780488
epoch: 1554, train_loss: 0.0007192898432865901, test_loss: 0.0006483441053812081
epoch: 1555, train_loss: 0.0007200161382333254, test_loss: 0.0006471253582276404
epoch: 1556, train_loss: 0.0007204311297518079, test_loss: 0.0006580036376059676
epoch: 1557, train_loss: 0.0007206564591995076, test_loss: 0.0006478885431230689
epoch: 1558, train_loss: 0.0007193864338383403, test_loss: 0.0006549836883399015
epoch: 1559, train_loss: 0.0007207081501331667, test_loss: 0.0006516615491515646
epoch: 1560, train_loss: 0.0007204259973545761, test_loss: 0.0006506770053723206
epoch: 1561, train_loss: 0.0007193686817671456, test_loss: 0.0006504226475954056
epoch: 1562, train_loss: 0.0007188644047583575, test_loss: 0.0006428450821355606
epoch: 1563, train_loss: 0.0007196417933775355, test_loss: 0.0006574398915593823
epoch: 1564, train_loss: 0.000720109231263885, test_loss: 0.0006464741066641485
epoch: 1565, train_loss: 0.000

In [ ]:
124/(199/39)

In [45]:
starting_epoch = 30
end_epoch = 1600
train_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), train_loss[starting_epoch:end_epoch])
test_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), test_loss[starting_epoch:end_epoch])
plt.legend([train_loss_curve, test_loss_curve], ['Train Loss', 'Validation Loss'])
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")

Text(0, 0.5, 'MSE Loss')

In [50]:
print("x mean:", x_mean, "; std:", x_std)
print("y mean:", y_mean, "; std:", y_std)
print("z mean:", z_mean, "; std:", z_std)

x mean: 0.5067009838472856 ; std: 0.2332880326514156
y mean: 0.49486543289715496 ; std: 0.23134116200489124
z mean: 0.4910597234688982 ; std: 0.23926200186407484


In [ ]:
test_pred

In [47]:
PATH = "model_1600.pt"
torch.save(model.state_dict(), PATH)

Loading Model

In [48]:
PATH = "model_1600.pt"
load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()
# mmodel = torch.load(PATH)

if torch.cuda.is_available():
    load_model.cuda()

In [49]:
# get test results
seq_dim = 10 # = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1
print(total_test_loss/test_batch)

0.0006484806799562648


In [52]:
for i, (seqs, labels) in enumerate(test_loader):
    print(seqs)

tensor([[[ 9.8338e-02,  2.5346e-04,  1.0032e+00],
         [ 1.3742e-01,  2.5346e-04,  1.0000e+00],
         [ 1.7205e-01,  2.5346e-04,  9.8299e-01],
         ...,
         [ 3.3839e-01,  2.5346e-04,  8.8670e-01],
         [ 3.7166e-01,  2.5346e-04,  8.6744e-01],
         [ 4.0493e-01,  2.5346e-04,  8.4819e-01]],

        [[ 1.3742e-01,  2.5346e-04,  1.0000e+00],
         [ 1.7205e-01,  2.5346e-04,  9.8299e-01],
         [ 2.0532e-01,  2.5346e-04,  9.6373e-01],
         ...,
         [ 3.7166e-01,  2.5346e-04,  8.6744e-01],
         [ 4.0493e-01,  2.5346e-04,  8.4819e-01],
         [ 4.3820e-01,  2.5346e-04,  8.2893e-01]],

        [[ 1.7205e-01,  2.5346e-04,  9.8299e-01],
         [ 2.0532e-01,  2.5346e-04,  9.6373e-01],
         [ 2.3859e-01,  2.5346e-04,  9.4447e-01],
         ...,
         [ 4.0493e-01,  2.5346e-04,  8.4819e-01],
         [ 4.3820e-01,  2.5346e-04,  8.2893e-01],
         [ 4.7147e-01,  2.5346e-04,  8.0967e-01]],

        ...,

        [[-2.6829e-01,  5.7823e-01, -1

In [ ]:
test_predd[-1]

In [ ]:
test_label_1

Analyze the distribution of ground true around predication

In [ ]:
# restore testing data
test_seq = []
test_gt = []
for i, (seqs, labels) in enumerate(test_loader):
    test_gt.append(labels)
    test_seq.append(seqs)

In [ ]:
for i in range(len(test_pred)):
    if (i == 0):
        test = test_seq[i].numpy()
        gt = test_gt[i].numpy()
        pred = test_pred[i].cpu().detach().numpy()
    else:
        test = np.append(test, test_seq[i].numpy(), axis = 0)
        gt = np.append(gt, test_gt[i].numpy(), axis = 0)
        pred = np.append(pred, test_pred[i].cpu().detach().numpy(), axis = 0)

In [ ]:
test.shape

In [ ]:
gt.shape

In [ ]:
pred.shape

In [ ]:
test[-1]

In [ ]:
gt[-1]

In [ ]:
pred[-1]

In [ ]:
d = test[0][-1,:].reshape(3, -1)

In [ ]:
test.shape

In [ ]:
d.shape

In [ ]:
gt[0].shape

In [ ]:
gt.shape

In [ ]:
print("x mean:", x_mean, "; std:", x_std)

Construct analysis matrix l(0-2 col): last sequence point, g(3-5 col): next ground true point, p(6-8 col): predicted point

In [ ]:
for i in range(test.shape[0]):
    last_point = test[i][-1, :].reshape(1, -1)
    gt_point = gt[i].reshape(1, -1)
    pred_point = pred[i].reshape(1, -1)
    row = np.append(np.append(last_point, gt_point, axis = 1), pred_point, axis = 1)
    if (i == 0):
        analysis = row
    else:
        analysis = np.append(analysis, row, axis = 0)
    

In [ ]:
# get rotation matrix
# note: input vec_1 and vec_2 have to be normalized before passing to the function
def get_rotatin_mat(vec_1, vec_2):
    a,b = vec_1.reshape(3), vec_2.reshape(3)
    v = np.cross(a,b)
    c = np.dot(a,b)
    s = np.linalg.norm(v)
    # print("s", s)
    if (s == 0):
        return np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
    I = np.identity(3)
    vXStr = '{} {} {}; {} {} {}; {} {} {}'.format(0, -v[2], v[1], v[2], 0, -v[0], -v[1], v[0], 0)
    k = np.matrix(vXStr)
    r = I + k + np.matmul(k,k) * ((1 -c)/(s**2))
    
    return r

In [ ]:
a = np.array([2, 1, 0])
a = a/np.linalg.norm(a)
b = np.array([4, 5, 6])
b = b/np.linalg.norm(b)
print("a", a)
print("b", b)
m = get_rotatin_mat(a, b)

In [ ]:
np.dot(m, a)

In [ ]:
m.shape

In [ ]:
np.sqrt(np.sum(np.square(b[:])))

In [ ]:
np.linalg.det(m)

In [ ]:
b/np.linalg.norm(b)

In [ ]:
a = np.array([[1,2,3]])
a_n = a/np.linalg.norm(a)

b = np.array([[4,5,6]])
b_n = b/np.linalg.norm(b)

c = np.array([[7,8,9]])
c_n = c/np.linalg.norm(c)

r = get_rotatin_mat(b_n, a_n) # b to a

b_p = np.dot(r, b.reshape(3, -1))
c_p = np.dot(r, c.reshape(3, -1))
c_p = np.asarray(c_p)
c_p_n = c_p/np.linalg.norm(c_p)
print(b.shape)
print("a", a)
print("b", b)
print("c", c)
print("b_p", b_p)
print("c_p", c_p)

print(get_rotatin_mat(c_n, b_n))

print(get_rotatin_mat(c_p_n, a_n))

In [ ]:
d = c_p.reshape(1, -1)
d = np.asarray(d)
d_n = d/np.linalg.norm(d)
d

In [ ]:
a

In [ ]:
d.shape

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
get_rotatin_mat(d_n, a)

In [ ]:
np.linalg.norm(b)

In [ ]:
np.linalg.norm(b_p)

In [ ]:
analysis.shape

In [ ]:
# combine all point to the same starting point
# first sequence last direction
# direction = (analysis[0, 0:3] - analysis[0, 3:6]).reshape(1, -1)
direction = (analysis[0, 3:6] - analysis[0, 0:3]).reshape(1, -1)
# normalize
direction = direction/np.linalg.norm(direction)
# first diff between prediction and ground true
diff_correct = (analysis[0, 6:9] - analysis[0, 3:6]).reshape(1, -1)
# print(type(diff_correct))
# print(diff_correct.shape)
for i in range(1, analysis.shape[0]):
    # cur_direction = (analysis[i, 0:3] - analysis[i, 3:6]).reshape(1, -1)
    cur_direction = (analysis[i, 3:6] - analysis[i, 0:3]).reshape(1, -1)
    # normalize
    if (np.linalg.norm(cur_direction) == 0):
        print(i, np.linalg.norm(cur_direction))
    cur_direction = cur_direction/np.linalg.norm(cur_direction)
    cur_diff = (analysis[i, 6:9] - analysis[i, 3:6]).reshape(1, -1)
    # get rotation matrix from cur_direction to direction
    r = get_rotatin_mat(cur_direction, direction)
    # apply rotation matrix to the cur_diff
    cur_diff = np.dot(r, cur_diff.reshape(3, -1))
    cur_diff = cur_diff.reshape(1, -1)
    # matrix type to np array type
    cur_diff = np.asarray(cur_diff)
    # print(cur_diff.shape)
    diff_correct = np.append(diff_correct, cur_diff, axis = 0)
    
    

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(diff_correct[:, 0], diff_correct[:, 1], diff_correct[:, 2])
x = [-direction[0, 0], 0]
y = [-direction[0, 1], 0]
z = [-direction[0, 2], 0]
ax.plot(x, y, z, label='parametric curve', color='r')
# ax.arrow(0, 0, 0.5, 0.5, head_width=0.05, head_length=0.1, fc='k', ec='k')
# ax.quiver(0, 0, 0, -direction[0, 0], -direction[0, 1], -direction[0, 2], length=5, normalize=True, color='r')
# x = np.zeros(10)
# y = np.zeros(10)
# z = np.arange(10)*10 # remove *100 and the arrow heads will reappear.
# dx = np.zeros(10)
# dy = np.arange(10)
# dz = np.zeros(10)
x = np.array([0, -direction[0, 0]])
y = np.array([0, -direction[0, 1]])
z = np.array([0, -direction[0, 2]])
dx = np.array([0, 0])
dy = np.array([0, 0])
dz = np.array([0, 0])
# ax.quiver(x, y, z, dx, dy, dz, length=1)
# ax.quiver(-direction[0, 0], -direction[0, 1], -direction[0, 2], 0, 0, 0, length=100, normalize=True)
ax.set_xlabel("x direction")
ax.set_ylabel("y direction")
ax.set_zlabel("z direction")
plt.show()



In [ ]:
direction

In [ ]:
np.zeros(10)

In [ ]:
view_angle = np.array([[1, 2, 3]])

In [ ]:
view_angle

In [ ]:
# note view_angle must be normalized
def getProjection(view_angle, point):
    v = point - np.array([[0, 0, 0]])
    dist = v[0, 0]*view_angle[0, 0] + v[0, 1]*view_angle[0, 1] + v[0, 2]*view_angle[0, 2]
    projected_point = point - dist*view_angle
    
    return projected_point

In [ ]:
diff_correct.shape

In [ ]:
direction[0, :].shape

In [ ]:
direction[0, :]

In [ ]:
view_angle = np.cross(direction[0, :].reshape(1, -1), np.array([[0, 0, 1]]))

In [ ]:
view_angle

In [ ]:
view_angle = np.cross(direction[0, :].reshape(1, -1), np.array([[0, 0, 1]]))
# normalization
view_angle = view_angle/np.linalg.norm(view_angle)
print(view_angle)
for i in range(diff_correct.shape[0]):
    if (i == 0):
        diff_corrrect_projection = getProjection(view_angle, diff_correct[i, :])
    else:
        diff_corrrect_projection = np.append(diff_corrrect_projection, getProjection(view_angle, diff_correct[i, :]), axis = 0)
    

In [ ]:
diff_corrrect_projection.shape

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(diff_corrrect_projection[:, 0], diff_corrrect_projection[:, 1], diff_corrrect_projection[:, 2])
x = [-direction[0, 0]*10, 0]
y = [-direction[0, 1]*10, 0]
z = [-direction[0, 2]*10, 0]
ax.plot(x, y, z, label='parametric curve', color='r')
ax.set_xlabel("x direction")
ax.set_ylabel("y direction")
ax.set_zlabel("z direction")
plt.show()

In [ ]:
diff_corrrect_projection

In [ ]:
# transform from 3D to 2D
diff_corrrect_projection_2D = np.delete(diff_corrrect_projection, 1, 1)

    

In [ ]:
from sklearn import mixture
from matplotlib.colors import LogNorm
clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(diff_corrrect_projection_2D)

In [ ]:
# display predicted scores by the model as a contour plot
x = np.linspace(-5., 5.)
y = np.linspace(-5., 5.)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 300))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(diff_corrrect_projection_2D[:, 0], diff_corrrect_projection_2D[:, 1], .8)

x = [-direction[0, 0]*5, 0]
z = [-direction[0, 2]*5, 0]
plt.plot(x, z, label='parametric curve', color='r')

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.xlabel("Direction 1")
plt.ylabel("Direction 2")
plt.show()

In [ ]:
n_samples = 300
C = np.array([[0., -0.7], [3.5, .7]])
stretched_gaussian = np.dot(np.random.randn(n_samples, 2), C)

In [ ]:
stretched_gaussian.shape

In [ ]:
diff_corrrect_projection_2D.shape

In [ ]:
direction

In [ ]:
pow(direction[0, 0], 2) + pow(direction[0, 1], 2) + pow(direction[0, 2], 2)

In [ ]:
analysis[2, 0:3] - analysis[2, 3:6]

In [ ]:
analysis[3, 0:3] - analysis[3, 3:6]

In [ ]:
np.linalg.norm([3, 4])

In [ ]:
direction = (analysis[0, 0:3] - analysis[0, 3:6]).reshape(1, -1)
direction = direction/np.linalg.norm(direction)

In [ ]:
(analysis[0, 0:3] - analysis[0, 3:6]).reshape(1, -1)

In [ ]:
(analysis[0, 0:3] - analysis[0, 3:6])

In [ ]:
d = (analysis[1, 0:3] - analysis[1, 3:6])

In [ ]:
d

In [ ]:
d/np.linalg.norm(d)

In [ ]:
pow(0.84804916, 2) + pow(0.5299187, 2)

In [ ]:
analysis[0:3]

In [ ]:
(analysis[0, 3:6] - analysis[0, 0:3])

In [ ]:
diff

In [ ]:
np.asarray([13.259178  ,  0.37397736, 20.36138]) - np.asarray([12.658086  ,  1.293065  ,20.224268])

In [ ]:
for i in range(len(test_pred)):
    if (i == 0):
        test = 
        gt = test_gt[i].cpu().detach().numpy()
        pred = test_pred[i].cpu().detach().numpy()
    else:
        gt = np.append(gt, test_gt[i].cpu().detach().numpy(), axis = 0)
        pred = np.append(pred, test_pred[i].cpu().detach().numpy(), axis = 0)

In [ ]:
gt.shape

In [ ]:
pred.shape

In [ ]:
diff = pred - gt

In [ ]:
diff

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(diff[:, 0], diff[:, 1], diff[:, 2])
plt.show()

In [ ]:
criterion(torch.from_numpy(gt), torch.from_numpy(pred))

In [ ]:
loss = 0
for i in range(len(test_pred)):
    l = criterion(test_gt[i], test_pred[i])
    loss += l

In [ ]:
loss